In [1]:
import requests
import time
import pandas as pd

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
BASE_URL = "https://ows.goszakup.gov.kz"
API_ENDPOINT = "/v3/rnu"
parquet_file = "goszakup_rnu_data.parquet"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  # Начинаем с первой страницы
all_data = []
seen_pids = set()
next_page = f"{API_ENDPOINT}?limit=500"  # Первоначальный запрос

while next_page:
    try:
        response = requests.get(f"{BASE_URL}{next_page}", headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Загрузка завершена, новых данных нет.")
            break

        # Фильтрация дубликатов
        new_items = [item for item in items if item["pid"] not in seen_pids]
        if not new_items:
            print("API вернуло дубликаты, остановка загрузки.")
            break

        all_data.extend(new_items)
        seen_pids.update(item["pid"] for item in new_items)

        print(f"Загружено {len(all_data)} записей...")

        # Получаем `next_page` из API
        next_page = data.get("next_page")
        if not next_page:
            print("Достигнута последняя страница, завершаем загрузку.")
            break

        # Промежуточное сохранение каждые 1000 записей
        if len(all_data) % 1000 < 500:
            pd.DataFrame(all_data).to_parquet(parquet_file, index=False, engine="pyarrow")
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
df = pd.DataFrame(all_data)
df.to_parquet(parquet_file, index=False, engine="pyarrow")

print(f"Данные сохранены в {parquet_file}")


Загружено 500 записей...
Загружено 1000 записей...
Промежуточное сохранение...
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out. (read timeout=30)
Ждём 30 сек и пробуем снова...
Загружено 1500 записей...
Загружено 2000 записей...
Промежуточное сохранение...
Загружено 2500 записей...
Загружено 3000 записей...
Промежуточное сохранение...
Загружено 3500 записей...
Загружено 4000 записей...
Промежуточное сохранение...
Загружено 4500 записей...
Загружено 5000 записей...
Промежуточное сохранение...
Загружено 5500 записей...
Загружено 6000 записей...
Промежуточное сохранение...
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out.
Ждём 30 сек и пробуем снова...
Загружено 6500 записей...
Загружено 7000 записей...
Промежуточное сохранение...
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out. (read timeout=30)
Ждём 30 сек и пробуем снова...
Загружено 7500 записей...
Загружено 800

In [2]:
df = pd.read_parquet("goszakup_rnu_data.parquet")
df.head()

,id,pid,supplier_biin,supplier_innunp,supplier_name_ru,supplier_name_kz,kato_list,system_id
0,242043,723601,240940016760,None,"ТОО ""Дария KZ""","""Дария KZ"" ЖШС",[391010000],3
1,242033,205744,920924400488,None,Amar ahlam,Amar ahlam,[591010000],3
2,242032,692022,491219400347,None,ЭлектроСтрой Астана,ЭлектроСтрой Астана,[151010000],3
3,242031,682888,981116300791,None,"""DEMURE""","""DEMURE""",[316235100],3
4,242022,719963,870304301615,None,Несибе ЕА,Несибе ЕА,[431010000],3


In [3]:
import numpy as np
df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

# Check for duplicates
duplicates = df.duplicated()
print(duplicates)

duplicates_count = df.duplicated().sum()
print(f'Количество дубликатов: {duplicates_count}')

C:\Users\alena\AppData\Local\Temp\ipykernel_2616\1402623328.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)


0        False
1        False
2        False
3        False
4        False
         ...  
17225    False
17226    False
17227    False
17228    False
17229    False
Length: 17230, dtype: bool
Количество дубликатов: 0
